In [1]:
from influxdb_client import InfluxDBClient

client = InfluxDBClient(
    url="http://localhost:8086",
    token="mytoken",
    org="scada"
)

health = client.health()
print("InfluxDB health:", health.status)


InfluxDB health: pass


In [26]:
from influxdb_client import InfluxDBClient
import pandas as pd

# -----------------------------
# InfluxDB configuration
# -----------------------------
INFLUXDB_URL = "http://localhost:8086"  # or your Docker container URL
INFLUXDB_TOKEN = "mytoken"
INFLUXDB_ORG = "scada"
INFLUXDB_BUCKET = "attack_data"

# -----------------------------
# Connect to InfluxDB
# -----------------------------
client = InfluxDBClient(
    url=INFLUXDB_URL,
    token=INFLUXDB_TOKEN,
    org=INFLUXDB_ORG
)

health = client.health()
print("InfluxDB health:", health.status)

# -----------------------------
# Simple query: last 10 records
# -----------------------------
query_api = client.query_api()
query = f'''
from(bucket: "{INFLUXDB_BUCKET}")
  |> range(start: 0)             // last 1 day
  |> filter(fn: (r) => r._measurement == "actuator_data")
  |> sort(columns: ["_stop"], desc: true)
  |> limit(n:5)
  |> pivot(
        rowKey: ["_time"],
        columnKey: ["_field"],
        valueColumn: "_value"
    )
|> drop(columns: ["_start","_stop","result","table","_measurement"])


'''

result = query_api.query(query=query)

# Convert to DataFrame
records = []
for table in result:
    for record in table.records:
        records.append(record.values)

df = pd.DataFrame(records)
print("Query result:")
df= df.head()
df.head()


InfluxDB health: pass
Query result:


,result,table,_time,P1_STATE,P2_STATE,P3_STATE,P4_STATE,P5_STATE,AIT 201,AIT 202,...,P501 Status,P502 Status,P6 STATE,P601 Status,P602 Status,P603 Status,PIT 501,PIT 502,PIT 503,UV401
0,_result,0,2019-07-20 08:22:36+00:00,2,2,10,4,12,128.524734,8.795053,...,2.0,1.0,2.0,1.0,1.0,1.0,158.485000,2.498959,113.368370,2.0
1,_result,1,2019-07-20 08:22:45+00:00,2,2,14,4,12,128.524734,8.793772,...,2.0,1.0,2.0,1.0,1.0,1.0,158.485000,2.498959,113.464493,2.0
2,_result,1,2019-07-20 08:22:48+00:00,2,2,14,4,12,128.524734,8.796976,...,2.0,1.0,2.0,1.0,1.0,1.0,158.725327,2.498959,113.464493,2.0
3,_result,1,2019-07-20 08:22:51+00:00,2,2,14,4,12,128.524734,8.800500,...,2.0,1.0,2.0,1.0,1.0,1.0,158.725327,2.498959,113.640732,2.0
4,_result,2,2019-07-20 08:22:53+00:00,2,2,15,4,12,128.524734,8.801461,...,2.0,1.0,2.0,1.0,1.0,1.0,158.725327,2.498959,113.640732,2.0


In [12]:
df.shape

(76, 81)

In [14]:

import os
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
BASE_DIR


NameError: name '__file__' is not defined